In [1]:
from astroquery.vizier import Vizier
import numpy as np
import pyds9
from glob import glob
from astropy.io import fits
from pyds9 import *
from astropy import wcs
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
from regions import read_ds9, write_ds9
from regions import DS9Parser
from regions import ds9_objects_to_string
from astropy.coordinates import Angle, SkyCoord
from regions import CircleSkyRegion

# This notebook will detect sources from a vizier catalogue  in a give file(s) using ds9.

<b> This notebook must be run off the NRAO server <b>

In [2]:
def pipeline(files,catalog='VIII/100/gleamegc'):
    Vizier.ROW_LIMIT = -1
    table = Vizier.get_catalogs(catalog='VIII/100/gleamegc')[0]
    Gleam_ra = table['RAJ2000']
    Gleam_dec = table['DEJ2000']
    Gleam_flux = table['Fintwide']
    
    #define a ra/dec mask so only sources in mosaics are chosen
    #I should edit this, seems like cuts arent very good...
    deccuts , racuts, fluxmin = [-41,-21] , [14,161] , 3 #deg,deg,Jy
    #create a mask for ra,dec and flux
    w = []
    for i in range(len(Gleam_dec)):
        w.append((deccuts[0] < Gleam_dec[i] < deccuts[1]) & (racuts[0] < Gleam_ra[i] < racuts[1]) & (Gleam_flux[i] > fluxmin))
 
    # #sources that fit our criteria
    data = table[w]
    Gleam_ra = data['RAJ2000']
    Gleam_dec = data['DEJ2000']
    Gleam_flux = data['Fintwide']
    
    
    # #open an instance of ds9
    ds9 = pyds9.DS9()
    
    for i in range(len(files)):
        print(files[i] + ' .... done')
        #opens wcs solutions in each header
        hdu = fits.open(files[i],mode='readonly', ignore_missing_end=True)
        imwcs = wcs.WCS(hdu[0].header)
        
        
        #loads each file into ds9 and circles GLEAM SOURCES
        ds9.set_pyfits(hdu)
        ds9.set('frame {}'.format(i+1))
        ds9.set('file {}'.format(files[i]))
        ds9.set('scale zscale')
        ds9.set('zoom to fit')
        
        # Applies wcs solutions to translate GLEAM positions to pixel values of fits image
        refx,refy = imwcs.wcs_world2pix(Gleam_ra[:], Gleam_dec[:],0)  
        for i in range(len(Gleam_dec)):
            ds9.set('regions', 'image; circle {},{} {} #color = green'.format(refx[i],refy[i], 15))
        
    return print('Success! Matched a total of {} sources!'.format(len(refx))) 

    

In [3]:
%%time
#here I specify the mosaiced fits files i want to use for the "pipeline"
files = sorted(glob('masterpmaps/*.fits'))

pipeline(files)

masterpmaps/masterpmap_2458098.fits .... done
masterpmaps/masterpmap_2458099.fits .... done
masterpmaps/masterpmap_2458101.fits .... done
masterpmaps/masterpmap_2458102.fits .... done
masterpmaps/masterpmap_2458103.fits .... done
masterpmaps/masterpmap_2458104.fits .... done
masterpmaps/masterpmap_2458105.fits .... done
masterpmaps/masterpmap_2458106.fits .... done
masterpmaps/masterpmap_2458107.fits .... done
masterpmaps/masterpmap_2458108.fits .... done
masterpmaps/masterpmap_2458109.fits .... done
masterpmaps/masterpmap_2458110.fits .... done
masterpmaps/masterpmap_2458111.fits .... done
masterpmaps/masterpmap_2458112.fits .... done
masterpmaps/masterpmap_2458113.fits .... done
masterpmaps/masterpmap_2458114.fits .... done
masterpmaps/masterpmap_2458115.fits .... done
masterpmaps/masterpmap_2458116.fits .... done
Success! Matched a total of 168 sources!
CPU times: user 2min 19s, sys: 982 ms, total: 2min 20s
Wall time: 2min 34s
